## Qwen3-4B-FP8 模型推理

In [1]:
# !uv pip install -U transformers

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 下载 Qwen/Qwen3-4B-FP8 模型，并从本地路径加载
MODEL_PATH = '../model/Qwen/Qwen3-4B-FP8'

加载 `tokenizer` 和 `model`

In [3]:
# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype="auto",
    device_map="auto"
)
print(f'model.device: {model.device}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model.device: cuda:0


通过 `tokenizer.apply_chat_template()` 对 `messages` 应用 Qwen3 聊天模板 `chat template`，将 `messages` 转成 Qwen3 模型认识的格式。

In [4]:
# prepare the model input
prompt = "目前比较主流的rag技术都有哪些"

# ChatML 格式，参考 https://docs.unsloth.ai/basics/datasets-guide#formatting-the-data
messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(f'model_inputs.keys(): {model_inputs.keys()}')

model_inputs.keys(): dict_keys(['input_ids', 'attention_mask'])


Qwen3 进行文本生成 (completion)

In [5]:
# model.generate?

In [6]:
# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.5
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
print(f'len(output_ids): {len(output_ids)}')

len(output_ids): 888


In [7]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

print(f'index: {index}')

index: 200


In [8]:
think_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
answer_content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("think_content:", think_content)
print("answer_content:", answer_content)

think_content: <think>
嗯，用户问的是现在市面上有哪些比较流行的RAG（Retrieval-Augmented Generation）技术和方法。首先我得回忆一下自己学过的相关知识。

记得以前在学习NLP的时候提到过一些基于检索增强生成的方法，在大模型中应用广泛吧？比如像DPR、T5这些可能属于基础框架？

然后还有各种不同的架构设计和优化策略呢！比如说多阶段召回系统或者混合搜索方式之类的扩展方案吗？
再想想最近有没有什么新的进展或变种形式出现啦～例如结合图神经网络进行更复杂的语义理解处理的情况也有听说哦！

另外还需要考虑不同应用场景下的选择差异性问题：是偏向于文档级别的信息整合还是需要跨模态的信息融合能力嘛～

对了还应该区分出开源实现与商业解决方案之间的区别，并且给出每个类型代表性的例子以及它们各自的特点优势所在。
这样整理下来就能形成一个结构清晰的回答给用户提供参考价值了吧~
</think>
answer_content: 当前业界广泛应用并持续演进的主要 **Relevance-Augmented Retrieval (RR)** 技术及其衍生方向可以分为以下几个核心类别：

---

### 一、**传统 RAG 基础范式**
1. **Document-Level Retriever + Generator 架构**

   - 核心组件：
     ```python
      # 检索器: DPR(Dense Passage Ranking)
       retriever = DensePassageRanking()
       
        generator= T5ForConditionalGeneration() 
         query="如何提高工作效率?"
          docs=retriever(query)  
           response_generator(generator,docs)

    ```
   
20世纪末到今早年间的典型流程包括三个步骤:
- 文本预训练(如BERT等Transformer系列);
- 索引构建(BM25/FAISS/Lucene etc.);
- 排序算法(SLAM/PQ/TensorFlow Serving).

> *注意

参考：

- Huggingface: [Qwen/Qwen3-4B-FP8](https://huggingface.co/Qwen/Qwen3-4B-FP8)
- Qwen blog: [Qwen3: Think Deeper, Act Faster](https://qwenlm.github.io/blog/qwen3/)